In [6]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("always")
import requests

In [7]:
data=pd.read_csv("HyderabadResturants.csv")
data.drop('links', axis=1, inplace = True)
data.head()

,names,ratings,cuisine,price for one
0,Sahara Bakers,3.7,"Chinese, Bakery, Sichuan, Pizza, Burger",100
1,KFC,3.9,"Burger, Fast Food, Biryani, Desserts, Beverages",100
2,Subbaiah Gari Hotel,4.1,"South Indian, Andhra, Mithai",100
3,Paradise Biryani,3.9,"Biryani, Kebab, Desserts, Beverages",100
4,Pista House Bakery,4.3,"Fast Food, Sandwich, Pizza, Burger, Wraps, Rol...",100


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 657 entries, 0 to 656
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   names          657 non-null    object
 1   ratings        657 non-null    object
 2   cuisine        657 non-null    object
 3   price for one  657 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 20.7+ KB


In [9]:
data.shape

(657, 4)

In [10]:
data[["cuisine1", "cuisine2",'cuisine3','cuisine4' ,'cuisine5','cuisine6','cuisine7','cuisine8']] = (  
    data["cuisine"].str.split(",", expand=True)     
)
data.rename(columns ={'price for one': 'price'},inplace=True)
data.head()

,names,ratings,cuisine,price,cuisine1,cuisine2,cuisine3,cuisine4,cuisine5,cuisine6,cuisine7,cuisine8
0,Sahara Bakers,3.7,"Chinese, Bakery, Sichuan, Pizza, Burger",100,Chinese,Bakery,Sichuan,Pizza,Burger,None,None,None
1,KFC,3.9,"Burger, Fast Food, Biryani, Desserts, Beverages",100,Burger,Fast Food,Biryani,Desserts,Beverages,None,None,None
2,Subbaiah Gari Hotel,4.1,"South Indian, Andhra, Mithai",100,South Indian,Andhra,Mithai,None,None,None,None,None
3,Paradise Biryani,3.9,"Biryani, Kebab, Desserts, Beverages",100,Biryani,Kebab,Desserts,Beverages,None,None,None,None
4,Pista House Bakery,4.3,"Fast Food, Sandwich, Pizza, Burger, Wraps, Rol...",100,Fast Food,Sandwich,Pizza,Burger,Wraps,Rolls,Salad,Desserts


In [11]:
features=["cuisine1","cuisine2","cuisine3","cuisine4","cuisine5","cuisine6","cuisine7","cuisine8"]
data["temp"]=data[features].isnull().sum(axis=1)
data["no_Of_cusines"]=8-data["temp"]
data.head()

,names,ratings,cuisine,price,cuisine1,cuisine2,cuisine3,cuisine4,cuisine5,cuisine6,cuisine7,cuisine8,temp,no_Of_cusines
0,Sahara Bakers,3.7,"Chinese, Bakery, Sichuan, Pizza, Burger",100,Chinese,Bakery,Sichuan,Pizza,Burger,None,None,None,3,5
1,KFC,3.9,"Burger, Fast Food, Biryani, Desserts, Beverages",100,Burger,Fast Food,Biryani,Desserts,Beverages,None,None,None,3,5
2,Subbaiah Gari Hotel,4.1,"South Indian, Andhra, Mithai",100,South Indian,Andhra,Mithai,None,None,None,None,None,5,3
3,Paradise Biryani,3.9,"Biryani, Kebab, Desserts, Beverages",100,Biryani,Kebab,Desserts,Beverages,None,None,None,None,4,4
4,Pista House Bakery,4.3,"Fast Food, Sandwich, Pizza, Burger, Wraps, Rol...",100,Fast Food,Sandwich,Pizza,Burger,Wraps,Rolls,Salad,Desserts,0,8


In [12]:
feature = data["cuisine"].tolist()
tfidf = text.TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(feature)
similarity = cosine_similarity(tfidf_matrix)

In [13]:
indices = pd.Series(data.index, index=data['names']).drop_duplicates()

In [14]:
def restaurant_recommendation(name, similarity = similarity):
    index = indices[name]
    similarity_scores = list(enumerate(similarity[index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[0:10]
    restaurantindices = [i[0] for i in similarity_scores]
    return data['names'].iloc[restaurantindices]

In [15]:
print(restaurant_recommendation("Subway"))

26                           Subway
87          FarmBowl By Urbankisaan
439        Juice And Sandwich Point
616                 Nature's Spirit
4                Pista House Bakery
293                          Qvinch
427    Gladiator Nutrition And Food
508                       Eggsplode
375                       Juice Box
54           Faasos - Wraps & Rolls
Name: names, dtype: object


In [16]:
response = requests.get('http://127.0.0.1:5000/status')
response.text

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /status (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000020DEBBD5B40>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
response = requests.get('http://127.0.0.1:5000/predict')
response.text

C:\Users\vamsi\AppData\Local\Temp\ipykernel_11552\2445697595.py:3: ResourceWarning: unclosed file <_io.BufferedWriter name='savemodel.sav'>
  pickle.dump(feature,open(file,'wb'))
